In [2]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
print(os.getcwd())


/Users/titoenriquez


In [6]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [12]:
##CODIGO DE RODRIGO
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
import time
import pandas as pd
import glob
import os

# Variables de apoyo
folder = os.path.abspath(os.curdir)
options = Options()
#options.add_argument("--headless") # Quitar # si quieres que no se abra un navegador externo
options.set_preference("browser.download.folderList", 2)
options.set_preference("browser.download.dir", folder)

with open('credentials.json','r') as f:
    """
    Si quieres automatizar la autenticación, ten un archivo "credentials.json" con tu usuario y contraseña.
    Si te parece palta o inseguro, reemplaza estas líneas de 'credentials' con tu acceso anterior.
    """
    
    credentials = eval(f.read())

with open('search.json','r') as f:
    """
    Para facilidad de trabajo, coloqué tus criterios de búsqueda en "search.json".
    Si prefieres tenerlo en tu código, reemplaza esto por un diccionario o lista de criterio.
    Ojo -sobre todo para daterange-, los formatos textuales deben escribirse tal como en mi referencia.
    """
    search_terms, daterange, pdf = eval(f.read()).values()

# Autenticación automática de parámetros mediante el método POST
url = f'https://up.idm.oclc.org/login?user={credentials['user']}&pass={credentials['pass']}&url=https://research.ebsco.com/c/avtvzl/search'
driver = webdriver.Firefox(options=options)
driver.get(url)
driver.maximize_window()

for term in search_terms:    
    """
    Lo siguiente es "ingeniería de parámetros URL" y "descarga en bulk de contenidos". 
    
    Cada componente después del "results?" corresponde a un parámetro del dominio. 
    Estos describen el comportamiento o el tipo de filtros de búsqueda que se ejecutarán en tu carga de página.
    La ingeniería de parámetros casi no interactúa con los objetos listados en la página, sino en cómo la página presenta sus contenidos.
    Así, evitamos la molestia de tener que introducir términos por el buscador.

    Una vez cargada la página, realizamos una vista de 50 contenidos por página y seleccionamos la opción nativa de descarga.
    De esta forma, te descargará los metadatps de los primeros 50 resultados más relevantes.
    Así, evitamos la molestia de seleccionar atributos específicos de cada contenido y que demore demasiado.
    """
    
    search = '&'.join([
        f'https://research-ebsco-com.up.idm.oclc.org/c/avtvzl/search/results?expanders=concept&{'expanders=fullText&' if pdf else ''}facetFilter=sourceTypes%3AMTYwTU4%3D', # Todo el URL y unos parámetros generales de búsqueda
        f'limiters=%3AY%2CDT1%3A{daterange[0]}%2F{daterange[1]}', # Parámetro de rango de tiempo
        'q=' + '%20'.join(term.split(' ')) # Parámetro de búsqueda
    ])

    driver.get(search)
    time.sleep(3)

    # Extensión a 50 contenidos por página. Por alguna razón, los dos pasos más importantes para que funcione.
    driver.find_element(By.XPATH,'//*[@id="results-per-page-dropdown-toggle-button"]').click()
    time.sleep(5)
    driver.find_element(By.XPATH, '//*[@id="results-per-page-dropdown-item-3"]').click()
    time.sleep(5)

    # Selección de los 50 contenidos.
    driver.find_element(By.ID, "downshift-0-toggle-button").click()
    time.sleep(5)
    driver.find_element(By.ID, "downshift-0-item-0").click()
    time.sleep(5)

    # Selección de tipo de descarga en formato .csv
    driver.find_element(By.XPATH,'//ul/li[1]/div/button').click()
    time.sleep(5)
    driver.find_element(By.XPATH, '//fieldset/div/span[3]/label/input').click()
    time.sleep(5)
    
    # Descarga
    driver.find_element(By.XPATH,'/html/body/div[19]/div/div/div[3]/button[2]').click()
    time.sleep(20)

# Por seguridad.
driver.quit()
del url, credentials, driver

# Selección de todos los archivos sueltos, unión y eliminación de duplicados.
archs = glob.glob(os.path.join(folder, "EBSCO-Metadata-*.csv"))
dfs = pd.concat([pd.read_csv(f) for f in archs], ignore_index=True)
dfs.drop_duplicates(inplace=True)
dfs.to_excel(f"resultados_ebsco_todos_{'bogota'}.xlsx", index=False)

# Borrar los archivos sueltos
for f in archs:
    try:
        os.remove(f)
    except Exception as e:
        print(f"⚠️ No se pudo eliminar {f}: {e}")


In [11]:
##MODIFICACION DE CHAT
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
import time
import pandas as pd
import glob
import os

# Variables de apoyo
folder = os.path.abspath(os.curdir)
options = Options()
#options.add_argument("--headless") # Quitar # si quieres que no se abra un navegador externo
options.set_preference("browser.download.folderList", 2)
options.set_preference("browser.download.dir", folder)

with open('credentials.json','r') as f:
    """
    Si quieres automatizar la autenticación, ten un archivo "credentials.json" con tu usuario y contraseña.
    Si te parece palta o inseguro, reemplaza estas líneas de 'credentials' con tu acceso anterior.
    """
    
    credentials = eval(f.read())

with open('search.json','r') as f:
    """
    Para facilidad de trabajo, coloqué tus criterios de búsqueda en "search.json".
    Si prefieres tenerlo en tu código, reemplaza esto por un diccionario o lista de criterio.
    Ojo -sobre todo para daterange-, los formatos textuales deben escribirse tal como en mi referencia.
    """
    search_terms, daterange, pdf = eval(f.read()).values()

# Autenticación automática de parámetros mediante el método POST
url = f'https://up.idm.oclc.org/login?user={credentials['user']}&pass={credentials['pass']}&url=https://research.ebsco.com/c/avtvzl/search'
driver = webdriver.Firefox(options=options)
driver.get(url)

for term in search_terms:    
    """
    Lo siguiente es "ingeniería de parámetros URL" y "descarga en bulk de contenidos". 
    
    Cada componente después del "results?" corresponde a un parámetro del dominio. 
    Estos describen el comportamiento o el tipo de filtros de búsqueda que se ejecutarán en tu carga de página.
    La ingeniería de parámetros casi no interactúa con los objetos listados en la página, sino en cómo la página presenta sus contenidos.
    Así, evitamos la molestia de tener que introducir términos por el buscador.

    Una vez cargada la página, realizamos una vista de 50 contenidos por página y seleccionamos la opción nativa de descarga.
    De esta forma, te descargará los metadatps de los primeros 50 resultados más relevantes.
    Así, evitamos la molestia de seleccionar atributos específicos de cada contenido y que demore demasiado.
    """
    
    search = '&'.join([
        f'https://research-ebsco-com.up.idm.oclc.org/c/avtvzl/search/results?expanders=concept&{'expanders=fullText&' if pdf else ''}facetFilter=sourceTypes%3AMTYwTU4%3D', # Todo el URL y unos parámetros generales de búsqueda
        f'limiters=%3AY%2CDT1%3A{daterange[0]}%2F{daterange[1]}', # Parámetro de rango de tiempo
        'q=' + '%20'.join(term.split(' ')) # Parámetro de búsqueda
    ])

    driver.get(search)
    time.sleep(3)

    # Extensión a 50 contenidos por página. Por alguna razón, los dos pasos más importantes para que funcione.
    driver.find_element(By.XPATH,'//*[@id="results-per-page-dropdown-toggle-button"]').click()
    time.sleep(3)
    driver.find_element(By.XPATH, '//*[@id="results-per-page-dropdown-item-3"]').click()
    time.sleep(3)

    # Selección de los 50 contenidos.
    driver.find_element(By.ID, "downshift-0-toggle-button").click()
    time.sleep(3)
    driver.find_element(By.ID, "downshift-0-item-0").click()
    time.sleep(3)

    # Selección de tipo de descarga en formato .csv
    driver.find_element(By.XPATH,'//ul/li[1]/div/button').click()
    time.sleep(5)
    driver.find_element(By.XPATH, '//fieldset/div/span[3]/label/input').click()
    time.sleep(5)
    
    # Descarga
    driver.find_element(By.XPATH,'/html/body/div[19]/div/div/div[3]/button[2]').click()
    time.sleep(20)

# Por seguridad.
driver.quit()
del url, credentials, driver

# Selección de todos los archivos sueltos, unión y eliminación de duplicados.
archs = glob.glob(os.path.join(folder, "EBSCO-Metadata-*.csv"))
dfs = pd.concat([pd.read_csv(f) for f in archs], ignore_index=True)
dfs.drop_duplicates(inplace=True)
dfs.to_excel("resultados_ebsco_todos.xlsx", index=False)

# Borrar los archivos sueltos
for f in archs:
    try:
        os.remove(f)
    except Exception as e:
        print(f"⚠️ No se pudo eliminar {f}: {e}")


NoSuchElementException: Message: Unable to locate element: [id="downshift-0-item-0"]; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:199:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:552:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16


In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
import time
import pandas as pd
import glob
import os

# Variables de apoyo
folder = os.path.abspath(os.curdir)
options = Options()
#options.add_argument("--headless") # Quitar # si quieres que no se abra un navegador externo
options.set_preference("browser.download.folderList", 2)
options.set_preference("browser.download.dir", folder)

with open('credentials.json','r') as f:
    """
    Si quieres automatizar la autenticación, ten un archivo "credentials.json" con tu usuario y contraseña.
    Si te parece palta o inseguro, reemplaza estas líneas de 'credentials' con tu acceso anterior.
    """
    
    credentials = eval(f.read())

with open('search.json','r') as f:
    """
    Para facilidad de trabajo, coloqué tus criterios de búsqueda en "search.json".
    Si prefieres tenerlo en tu código, reemplaza esto por un diccionario o lista de criterio.
    Ojo -sobre todo para daterange-, los formatos textuales deben escribirse tal como en mi referencia.
    """
    search_terms, daterange, pdf = eval(f.read()).values()

# Autenticación automática de parámetros mediante el método POST
url = f'https://up.idm.oclc.org/login?user={credentials['user']}&pass={credentials['pass']}&url=https://research.ebsco.com/c/avtvzl/search'
driver = webdriver.Firefox(options=options)
driver.get(url)

for term in search_terms:    
    """
    Lo siguiente es "ingeniería de parámetros URL" y "descarga en bulk de contenidos". 
    
    Cada componente después del "results?" corresponde a un parámetro del dominio. 
    Estos describen el comportamiento o el tipo de filtros de búsqueda que se ejecutarán en tu carga de página.
    La ingeniería de parámetros casi no interactúa con los objetos listados en la página, sino en cómo la página presenta sus contenidos.
    Así, evitamos la molestia de tener que introducir términos por el buscador.

    Una vez cargada la página, realizamos una vista de 50 contenidos por página y seleccionamos la opción nativa de descarga.
    De esta forma, te descargará los metadatps de los primeros 50 resultados más relevantes.
    Así, evitamos la molestia de seleccionar atributos específicos de cada contenido y que demore demasiado.
    """
    
    search = '&'.join([
        f'https://research-ebsco-com.up.idm.oclc.org/c/avtvzl/search/results?expanders=concept&{'expanders=fullText&' if pdf else ''}facetFilter=sourceTypes%3AMTYwTU4%3D', # Todo el URL y unos parámetros generales de búsqueda
        f'limiters=%3AY%2CDT1%3A{daterange[0]}%2F{daterange[1]}', # Parámetro de rango de tiempo
        'q=' + '%20'.join(term.split(' ')) # Parámetro de búsqueda
    ])

    driver.get(search)
    time.sleep(3)

    # Extensión a 50 contenidos por página. Por alguna razón, los dos pasos más importantes para que funcione.
    driver.find_element(By.XPATH,'//*[@id="results-per-page-dropdown-toggle-button"]').click()
    time.sleep(3)
    driver.find_element(By.XPATH, '//*[@id="results-per-page-dropdown-item-3"]').click()
    time.sleep(3)

    # Selección de los 50 contenidos.
    driver.find_element(By.ID, "downshift-0-toggle-button").click()
    time.sleep(3)
    driver.find_element(By.ID, "downshift-0-item-0").click()
    time.sleep(3)

    # Selección de tipo de descarga en formato .csv
    driver.find_element(By.XPATH,'//ul/li[1]/div/button').click()
    time.sleep(5)
    driver.find_element(By.XPATH, '//fieldset/div/span[3]/label/input').click()
    time.sleep(5)
    
    # Descarga
    driver.find_element(By.XPATH,'/html/body/div[19]/div/div/div[3]/button[2]').click()
    time.sleep(20)

# Por seguridad.
driver.quit()
del url, credentials, driver

# Selección de todos los archivos sueltos, unión y eliminación de duplicados.
archs = glob.glob(os.path.join(folder, "EBSCO-Metadata-*.csv"))
dfs = pd.concat([pd.read_csv(f) for f in archs], ignore_index=True)
dfs.drop_duplicates(inplace=True)
dfs.to_excel("resultados_ebsco_todos.xlsx", index=False)

# Borrar los archivos sueltos
for f in archs:
    try:
        os.remove(f)
    except Exception as e:
        print(f"⚠️ No se pudo eliminar {f}: {e}")


ElementClickInterceptedException: Message: Element <li id="results-per-page-dropdown-item-3" class="eb-dropdown__item dropdown__item results-per-page-dropdown_results-per-page-dropdown__ojUkW__item eb-dropdown__item--standard dropdown__item--standard results-per-page-dropdown_results-per-page-dropdown__ojUkW__item--standard"> is not clickable at point (977,457) because another element <div id="c271f492-9c2e-481f-a2a8-be7dd2c93a23" class="osano-cm-window__dialog osano-cm-dialog osano-cm-dialog--position_bottom-right osano-cm-dialog--type_box"> obscures it
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:199:5
ElementClickInterceptedError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:351:5
webdriverClickElement@chrome://remote/content/marionette/interaction.sys.mjs:177:11
interaction.clickElement@chrome://remote/content/marionette/interaction.sys.mjs:136:11
clickElement@chrome:/